In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.cost_function import SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.genetic_algorithm.simple_genetic_algorithm import (
    SimpleGeneticAlgorithm,
    SimpleGeneticAlgorithmParameters,
)
from seapopym_optimization.genetic_algorithm.simple_logbook import Logbook
from seapopym_optimization.model_generator import NoTransportModelGenerator


/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.13/site-packages/seapopym/standard/units.py:9: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  import cf_xarray.units  # noqa: F401


In [2]:
import logging

logging.basicConfig(level=logging.WARNING, format="\n%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("seapopym_optimization")
logger.setLevel(logging.DEBUG)

In [3]:
fg1 = NoTransportFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

fg2 = NoTransportFunctionalGroup(
    name="FG2",
    day_layer=1,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

In [4]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_transfert", "FG2_energy_transfert"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [5]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)

In [6]:
fg_set.unique_functional_groups_parameters_ordered().keys()

dict_keys(['FG1_energy_transfert', 'lambda_temperature_0', 'gamma_lambda_temperature', 'tr_0', 'gamma_tr', 'FG2_energy_transfert'])

In [7]:
parameters_set = [1, np.inf, 1, 1, -1, 1]
functional_group_parameters = fg_set.generate(parameters_set)
functional_group_parameters

[{'day_layer': 0,
  'night_layer': 0,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1},
 {'day_layer': 1,
  'night_layer': 1,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1}]

In [8]:
nb_days = 360
temperature = xr.DataArray(
    data=np.zeros((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=np.ones((nb_days, 1, 1)),
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "kg/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
observation = xr.DataArray(
    data=np.ones((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "layer"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "layer": [0, 1],
    },
    attrs={
        "units": "kg/m^2",
    },
)
observation.time.attrs = {"axis": "T"}
temperature.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}

observation.latitude.attrs = {"axis": "Y"}
temperature.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}

observation.longitude.attrs = {"axis": "X"}
temperature.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

observation.layer.attrs = {"axis": "Z"}
temperature.depth.attrs = {"axis": "Z"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
)
with xr.set_options(keep_attrs=True):
    observation_2 = observation.sel(layer=[1]) / 2

In [9]:
observation_1 = TimeSeriesObservation(
    name="Observation_all",
    observation=observation,
)
observation_2 = TimeSeriesObservation(
    name="Observation_meso",
    observation=observation_2,
)

In [10]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameter)

In [11]:
model = model_generator.generate(functional_group_parameters=functional_group_parameters)

In [12]:
model.state

<xarray.Dataset> Size: 12kB
Dimensions:                     (time: 360, latitude: 1, longitude: 1,
                                 depth: 2, functional_group: 2, cohort: 1)
Coordinates:
  * latitude                    (latitude) int64 8B 0
  * longitude                   (longitude) int64 8B 0
  * depth                       (depth) int64 16B 0 1
  * time                        (time) datetime64[ns] 3kB 2023-01-01 ... 2023...
  * cohort                      (cohort) int64 8B 0
  * functional_group            (functional_group) int64 16B 0 1
Data variables: (12/18)
    temperature                 (time, latitude, longitude, depth) float64 6kB ...
    primary_production          (time, latitude, longitude) float64 3kB 1.0 ....
    name                        (functional_group) <U7 56B 'Group_0' 'Group_1'
    energy_transfert            (functional_group) int64 16B 1 1
    lambda_temperature_0        (functional_group) float64 16B inf inf
    gamma_lambda_temperature    (functional_group) int64 16B 1 1
    ...                          ...
    max_timestep                (functional_group, cohort) int64 16B 1 1
    mean_timestep               (functional_group, cohort) float64 16B 1.0 1.0
    timestep                    int64 8B 1
    angle_horizon_sun           float64 8B 0.0
    compute_initial_conditions  bool 1B False
    compute_preproduction       bool 1B False

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[observation_1, observation_2],
    functional_groups=[fg1, fg2],
    root_mse=True,
    centered_mse=False,
    normalized_mse=False,
)
partial_cost_function = cost_function.generate()
partial_cost_function(parameters_set)


(np.float64(0.0), np.float64(0.5))

In [14]:
metaparam = SimpleGeneticAlgorithmParameters(
    ETA=4, INDPB=0.2, CXPB=0.5, MUTPB=0.2, NGEN=3, POP_SIZE=10, cost_function_weight=(-1, -1)
)
genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam, cost_function=cost_function, constraint=[constraint]
)


In [15]:
genetic_algorithm.optimize()

SimpleViewer(logbook=category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.302013   
                                       1                      0.178207   
                                       2                      0.691410   
                                       3                      0.706082   
                                       4                      0.084111   
                                       5                      0.539268   
                                       6                      0.252048   
                                       7                      0.822050   
                                       8                      0.025488   
                                       9                      0.828567   
1          False 

In [16]:
genetic_algorithm.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.302013   
                                       1                      0.178207   
                                       2                      0.691410   
                                       3                      0.706082   
                                       4                      0.084111   
                                       5                      0.539268   
                                       6                      0.252048   
                                       7                      0.822050   
                                       8                      0.025488   
                                       9                      0.828567   
1          False                       0                      0.252048   
           True                        1                      0.539268   
           False                       2                      0.252048   
                                       3                      0.252048   
                                       4                      0.025488   
                                       5                      0.252048   
                                       6                      0.025488   
                                       7                      0.539268   
                                       8                      0.252048   
                                       9                      0.000636   
2          False                       0                      0.539268   
                                       1                      0.252048   
                                       2                      0.252048   
                                       3                      0.539268   
                                       4                      0.252048   
                                       5                      0.539268   
           True                        6                      0.252048   
                                       7                      0.539268   
           False                       8                      0.252048   
                                       9                      0.252048   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     20.239979   
                                       1                     41.238219   
                                       2                     47.709102   
                                       3                     55.420752   
                                       4                     51.232934   
                                       5                     11.308342   
                                       6                     97.825262   
                                       7                     33.168807   
                                       8                      5.133500   
                                       9                     52.977932   
1          False                       0                     97.825262   
           True                        1                     11.308342   
           False                       2                     97.825262   
                                       3                     97.825262   
                                       4                      5.133500   
                                       5                     97.825262   
                                       6                      5.133500   
                                       7                     11.308342   
   

In [17]:
unevaluated_logbook = genetic_algorithm.logbook.loc[[0]].copy()
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.302013   
                                       1                      0.178207   
                                       2                      0.691410   
                                       3                      0.706082   
                                       4                      0.084111   
                                       5                      0.539268   
                                       6                      0.252048   
                                       7                      0.822050   
                                       8                      0.025488   
                                       9                      0.828567   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     20.239979   
                                       1                     41.238219   
                                       2                     47.709102   
                                       3                     55.420752   
                                       4                     51.232934   
                                       5                     11.308342   
                                       6                     97.825262   
                                       7                     33.168807   
                                       8                      5.133500   
                                       9                     52.977932   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.549872   
                                       1                          0.989023   
                                       2                          0.887532   
                                       3                          0.795424   
                                       4                          0.561791   
                                       5                          0.645883   
                                       6                          0.123494   
                                       7                          0.517690   
                                       8                          0.895920   
                                       9                          0.651283   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           2.907506 -0.839637   
                                       1           7.394048 -0.908336   
                                       2           3.787136 -0.925449   
                                       3           4.963653 -0.470317   
                                       4           3.165215 -0.730666   
                                       5           8.658825 -0.236427   
                                       6           7.981576 -0.291258   
                                       7           2.862528 -0.000860   
                                       8           3.569384 -0.175495   
                                       9           5.000212 -0.769159   

category                                                                \
name                               

In [18]:
unevaluated_logbook.iloc[:5, -3:] = np.nan
unevaluated_logbook = Logbook(unevaluated_logbook)
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.302013   
                                       1                      0.178207   
                                       2                      0.691410   
                                       3                      0.706082   
                                       4                      0.084111   
                                       5                      0.539268   
                                       6                      0.252048   
                                       7                      0.822050   
                                       8                      0.025488   
                                       9                      0.828567   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     20.239979   
                                       1                     41.238219   
                                       2                     47.709102   
                                       3                     55.420752   
                                       4                     51.232934   
                                       5                     11.308342   
                                       6                     97.825262   
                                       7                     33.168807   
                                       8                      5.133500   
                                       9                     52.977932   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.549872   
                                       1                          0.989023   
                                       2                          0.887532   
                                       3                          0.795424   
                                       4                          0.561791   
                                       5                          0.645883   
                                       6                          0.123494   
                                       7                          0.517690   
                                       8                          0.895920   
                                       9                          0.651283   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           2.907506 -0.839637   
                                       1           7.394048 -0.908336   
                                       2           3.787136 -0.925449   
                                       3           4.963653 -0.470317   
                                       4           3.165215 -0.730666   
                                       5           8.658825 -0.236427   
                                       6           7.981576 -0.291258   
                                       7           2.862528 -0.000860   
                                       8           3.569384 -0.175495   
                                       9           5.000212 -0.769159   

category                                                                \
name                               

In [19]:
new_genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam,
    cost_function=cost_function,
    constraint=[constraint],
    logbook=unevaluated_logbook,
)


In [27]:
viewer = new_genetic_algorithm.optimize()


2025-06-06 16:28:24,526 - seapopym_optimization.genetic_algorithm.simple_genetic_algorithm - INFO - Logbook found. Loading last generation.


In [29]:
viewer.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.302013   
                                       1                      0.178207   
                                       2                      0.691410   
                                       3                      0.706082   
                                       4                      0.084111   
           True                        5                      0.539268   
                                       6                      0.252048   
                                       7                      0.822050   
                                       8                      0.025488   
                                       9                      0.828567   
1          True                        0                      0.302013   
                                       1                      0.178207   
                                       2                      0.084111   
           False                       3                      0.324992   
                                       4                      0.084111   
           True                        5                      0.822050   
                                       6                      0.084111   
                                       7                      0.178207   
                                       8                      0.178207   
           False                       9                      0.025488   
2          True                        0                      0.178207   
           False                       1                      0.324992   
                                       2                      0.084111   
                                       3                      0.178207   
                                       4                      0.324992   
                                       5                      0.178207   
           True                        6                      0.178207   
                                       7                      0.178207   
                                       8                      0.178207   
                                       9                      0.324992   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                     20.239979   
                                       1                     41.238219   
                                       2                     47.709102   
                                       3                     55.420752   
                                       4                     51.232934   
           True                        5                     11.308342   
                                       6                     97.825262   
                                       7                     33.168807   
                                       8                      5.133500   
                                       9                     52.977932   
1          True                        0                     20.239979   
                                       1                     41.238219   
                                       2                     51.232934   
           False                       3                     97.825262   
                                       4                     66.556678   
           True                        5                     33.168807   
                                       6                     51.232934   
                                       7                     41.238219   
   

In [30]:
viewer.stats

,mean,std,min,max,valid,from_previous_generation
Generation,,,,,,
0,-0.498115,0.099139,-0.611006,-0.372961,4,0.500
1,-0.530722,0.087891,-0.611006,-0.351002,8,0.625
2,-0.482742,0.088003,-0.612810,-0.346965,10,0.500
